This notebook contains the data cleaning of the metrics chosen for Part 2 Analysis. These metrics are:
1. Income and Industry data for Salina
2. Unemployment and employment data for Salina

# Section 1: Importing necessary modules

In [26]:
import pandas as pd
import numpy as np
import glob
from scipy.optimize import nnls
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Section 2: Cleaning Income and Industry data for Salina 

The data resides in twelve csv files named 2010_Census.csv, 2011_Census.csv,,..., 2021_Census.csv. We iterate through each of these and perform appropriate data cleaning.  

This step also involves removing all columns with '(X)' values or all null values, and those starting with 'Percent' or 'Unnamed'. Then, we select the columns related to Income and Industry by searching for these strings within the columns and finally, transposing the dataframes to then add Metrics and Sub Metric columns. These two new columns are then populated with the appropriate values (Metric - either Income or Industry, Submetric - different classes of Income/Industry).   

Finally, we rename and drop unnecessary columns and then set the corresponfing year.

In [27]:
def import_clean_data(year):
    file_path = f'/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/data_sources/{year}_Census.csv'
    data = pd.read_csv(file_path, header=1)
    
    # Data cleaning steps
    data = data.drop(['Geography', 'Geographic Area Name'], axis=1)

    # Remove columns with all '(X)' values or all null values
    columns_to_drop_1 = data.columns[data.eq('(X)').all()]
    columns_to_drop_2 = data.columns[data.isnull().all()]
    data.drop(columns=columns_to_drop_1, inplace=True)
    data.drop(columns=columns_to_drop_2, inplace=True)

    # Drop columns starting with 'Percent' or 'Unnamed:'
    columns_to_drop_3 = [col for col in data.columns if col.startswith(('Percent', 'Unnamed:'))]
    data.drop(columns=columns_to_drop_3, inplace=True)

    # Select columns related to 'INCOME' or 'INDUSTRY'
    columns_to_keep = [col for col in data.columns if 'INCOME' in col or 'INDUSTRY' in col]
    data = data[columns_to_keep]

    # Transpose the DataFrame, add 'Metrics', 'Sub Metric' columns
    data = data.T
    data['Metrics'] = np.nan
    data['Sub Metric'] = np.nan
    data = data.reset_index()

    # Classify 'Metrics' and 'Sub Metric' based on column names
    for col in data['index']:
        if 'INDUSTRY' in col:
            data.loc[data['index'] == col, 'Metrics'] = 'INDUSTRY'
            data.loc[data['index'] == col, 'Sub Metric'] = col.rsplit("!!", 1)[-1]
        elif 'INCOME' in col:
            data.loc[data['index'] == col, 'Metrics'] = 'INCOME'
            data.loc[data['index'] == col, 'Sub Metric'] = col.rsplit("!!", 1)[-1]

    # Rename columns, drop unnecessary columns, and set corr year
    data = data.rename(columns={0: 'Salina_Count'})
    data = data.drop('index', axis=1)
    data['Year'] = year
    data.drop_duplicates(subset='Sub Metric', keep='first', inplace=True)

    return data

We create df_income_industry, which is now the final dataframe containing values for income and employment industry in the city of salina from years 2011 to 2021.

In [28]:
# Create an empty DataFrame to store concatenated data
df_income_industry = pd.DataFrame()

# Loop through years from 2011 to 2021 and process data
for year in range(2011, 2022):
    data_year = import_clean_data(year)
    data_name = f'data_{year}'
    globals()[data_name] = data_year  # Create individual DataFrames for each year
    df_income_industry = pd.concat([df_income_industry, data_year])  # Concatenate data

# Reset index for the combined DataFrame
df_income_industry = df_income_industry.reset_index(drop=True)

In [29]:
df_income_industry

,Salina_Count,Metrics,Sub Metric,Year
0,24508,INDUSTRY,Civilian employed population 16 years and over,2011
1,185,INDUSTRY,"Agriculture, forestry, fishing and hunting, an...",2011
2,1199,INDUSTRY,Construction,2011
3,4507,INDUSTRY,Manufacturing,2011
4,722,INDUSTRY,Wholesale trade,2011
...,...,...,...,...
543,30282,INCOME,Median nonfamily income (dollars),2021
544,38675,INCOME,Mean nonfamily income (dollars),2021
545,31355,INCOME,Median earnings for workers (dollars),2021
546,46033,INCOME,"Median earnings for male full-time, year-round...",2021


In [30]:
df_income_industry.to_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/intermediate_data/income_industry.csv')

In [25]:
# Checking to see if there are any null values
null_count = df_income_industry.isnull().sum()
non_null_count = df_income_industry.notnull().sum()
total_count = df_income_industry.count()

count_summary = pd.DataFrame({
    'Null Values': null_count,
    'Non-Null Values': non_null_count,
    'Total Values': total_count
})
print(count_summary)

              Null Values  Non-Null Values  Total Values
Salina_Count            0              548           548
Metrics                 0              548           548
Sub Metric              0              548           548
Year                    0              548           548


# Section 3: Cleaning Employment and Unemployment data for Salina

We firstly import the employment data for Salina and remove the unnecessary columns and only retain year and count of employment in the city.

In [31]:
# Importing the Employment data
df_employment = pd.read_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/data_sources/Salina_Monthly_Employment.csv')
df_employment

,Series ID,Year,Period,Label,Value
0,LAUCT206270000000005,1990,M01,1990 Jan,21711
1,LAUCT206270000000005,1990,M02,1990 Feb,21308
2,LAUCT206270000000005,1990,M03,1990 Mar,21543
3,LAUCT206270000000005,1990,M04,1990 Apr,21660
4,LAUCT206270000000005,1990,M05,1990 May,21816
...,...,...,...,...,...
400,LAUCT206270000000005,2023,M05,2023 May,24859
401,LAUCT206270000000005,2023,M06,2023 Jun,24617
402,LAUCT206270000000005,2023,M07,2023 Jul,25087
403,LAUCT206270000000005,2023,M08,2023 Aug,25159


In [32]:
df_employment = df_employment.drop(['Series ID', 'Label','Period'], axis=1)
df_employment = df_employment.groupby('Year').agg({'Value': 'mean'}).reset_index()
df_employment = df_employment.rename(columns={'Value': 'Salina_Employment'})

In [33]:
df_employment.head()

,Year,Salina_Employment
0,1990,22082.666667
1,1991,22907.333333
2,1992,23572.000000
3,1993,23461.000000
4,1994,23131.250000


Similarly, we import the unemployment data for Salina and remove the unnecessary columns and only retain year and count of unemployment in the city.

In [34]:
# Importing the Unemployment data
df_unemployment = pd.read_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/data_sources/Salina_Monthly_Unemployment.csv')

df_unemployment = df_unemployment.drop(['Series ID', 'Label','Period'], axis=1)
df_unemployment = df_unemployment.groupby('Year').agg({'Value': 'mean'}).reset_index()
df_unemployment = df_unemployment.rename(columns={'Value': 'Salina_Unemployment'})

In [35]:
df_unemployment.head()

,Year,Salina_Unemployment
0,1990,902.250000
1,1991,921.083333
2,1992,914.666667
3,1993,1010.333333
4,1994,845.833333


# Section 4: Merging both the employment and unemployment data

In [36]:
df_employment_unemployment = pd.merge(df_employment, df_unemployment, on='Year', how='inner')
df_employment_unemployment.head()

,Year,Salina_Employment,Salina_Unemployment
0,1990,22082.666667,902.250000
1,1991,22907.333333,921.083333
2,1992,23572.000000,914.666667
3,1993,23461.000000,1010.333333
4,1994,23131.250000,845.833333


df_employment_unemployment contains the final data for unemployment and unemployment for Salina

# Section 5: Final Analysis Datasets

In [38]:
df_smoke=pd.read_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/intermediate_data/pred_df.csv')

In [39]:
df_smoke_pred_1 = pd.merge(df_employment_unemployment, df_smoke, on='Year', how='inner')
df_smoke_pred_1 = df_smoke_pred_1.drop(['shortest_dist','GISAcres'], axis=1)
df_smoke_pred_1.head()

,Year,Salina_Employment,Salina_Unemployment,Smoke_Estimate,AQI
0,1990,22082.666667,902.250000,0.120077,8.809696
1,1991,22907.333333,921.083333,0.383275,8.814123
2,1992,23572.000000,914.666667,0.180287,8.818550
3,1993,23461.000000,1010.333333,0.264069,8.822977
4,1994,23131.250000,845.833333,0.421309,8.827404


In [40]:
df_smoke_pred_1.to_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/intermediate_data/analysis_1.csv', index=False)

In [41]:
df_smoke_pred_2 = pd.merge(df_income_industry, df_smoke, on='Year', how='inner')
df_smoke_pred_2 = df_smoke_pred_2.drop(['shortest_dist','GISAcres'], axis=1)
df_smoke_pred_2.head()

,Salina_Count,Metrics,Sub Metric,Year,Smoke_Estimate,AQI
0,24508,INDUSTRY,Civilian employed population 16 years and over,2011,1.759057,26.842567
1,185,INDUSTRY,"Agriculture, forestry, fishing and hunting, an...",2011,1.759057,26.842567
2,1199,INDUSTRY,Construction,2011,1.759057,26.842567
3,4507,INDUSTRY,Manufacturing,2011,1.759057,26.842567
4,722,INDUSTRY,Wholesale trade,2011,1.759057,26.842567


In [42]:
df_smoke_pred_2.to_csv('/Users/aviva/Desktop/HCDS Project/salina_wildfire_analysis/intermediate_data/analysis_2.csv', index=False)